<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="cognitiveclass.ai logo">
</center>


# Machine Learning Foundation

## Course 4, Part e: Non-Negative Matrix Factorization DEMO


This exercise illustrates usage of Non-negative Matrix factorization and covers techniques related to sparse matrices and some basic work with Natural Langauge Processing.  We will use NMF to look at the top words for given topics.


## Data


We'll be using the BBC dataset. These are articles collected from 5 different topics, with the data pre-processed. 

These data are available in the data folder (or online [here](http://mlg.ucd.ie/files/datasets/bbc.zip?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01)). The data consists of a few files. The steps we'll be following are:

* *bbc.terms* is just a list of words 
* *bbc.docs* is a list of artcles listed by topic.

At a high level, we're going to 

1. Turn the `bbc.mtx` file into a sparse matrix (a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01) format can be useful for matrices with many values that are 0, and save space by storing the position and values of non-zero elements).
1. Decompose that sparse matrix using NMF.
1. Use the resulting components of NMF to analyze the topics that result.


## Data Setup


Note: This lab has been updated to work in skillsnetwork for your convenience.


In [22]:
import urllib

In [23]:
with urllib.request.urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/bbc.mtx') as r:
    content = r.readlines()[2:]

In [24]:
content[0:10]

[b'1 1 1.0\n',
 b'1 7 2.0\n',
 b'1 11 1.0\n',
 b'1 14 1.0\n',
 b'1 15 2.0\n',
 b'1 19 2.0\n',
 b'1 21 1.0\n',
 b'1 29 1.0\n',
 b'1 30 1.0\n',
 b'1 33 1.0\n']

## Part 1

Here, we will turn this into a list of tuples representing a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01). Remember the description of the file from above:

* *bbc.mtx* is a list: first column is **wordID**, second is **articleID** and the third is the number of times that word appeared in that article.

So, if word 1 appears in article 3, 2 times, one element of our list will be:

`(1, 3, 2)`


In [25]:
sparsemat = [tuple(map(int,map(float,c.split()))) for c in content]
# Let's examine the first few elements
sparsemat[:8]

[(1, 1, 1),
 (1, 7, 2),
 (1, 11, 1),
 (1, 14, 1),
 (1, 15, 2),
 (1, 19, 2),
 (1, 21, 1),
 (1, 29, 1)]

In [73]:
sparsemat

[(1, 1, 1),
 (1, 7, 2),
 (1, 11, 1),
 (1, 14, 1),
 (1, 15, 2),
 (1, 19, 2),
 (1, 21, 1),
 (1, 29, 1),
 (1, 30, 1),
 (1, 33, 1),
 (1, 35, 1),
 (1, 41, 1),
 (1, 45, 1),
 (1, 47, 2),
 (1, 50, 1),
 (1, 52, 1),
 (1, 53, 3),
 (1, 55, 1),
 (1, 61, 1),
 (1, 62, 1),
 (1, 63, 1),
 (1, 65, 1),
 (1, 69, 1),
 (1, 80, 1),
 (1, 81, 1),
 (1, 86, 1),
 (1, 93, 1),
 (1, 99, 1),
 (1, 100, 1),
 (1, 104, 1),
 (1, 105, 1),
 (1, 106, 1),
 (1, 112, 1),
 (1, 116, 3),
 (1, 117, 1),
 (1, 118, 2),
 (1, 120, 1),
 (1, 121, 1),
 (1, 126, 1),
 (1, 131, 4),
 (1, 133, 1),
 (1, 134, 3),
 (1, 138, 2),
 (1, 140, 2),
 (1, 143, 1),
 (1, 151, 2),
 (1, 152, 2),
 (1, 175, 1),
 (1, 176, 1),
 (1, 177, 1),
 (1, 180, 1),
 (1, 184, 1),
 (1, 189, 1),
 (1, 194, 1),
 (1, 195, 1),
 (1, 198, 1),
 (1, 201, 1),
 (1, 206, 1),
 (1, 207, 3),
 (1, 208, 1),
 (1, 217, 1),
 (1, 219, 1),
 (1, 221, 1),
 (1, 231, 1),
 (1, 241, 2),
 (1, 252, 1),
 (1, 253, 2),
 (1, 255, 2),
 (1, 263, 1),
 (1, 273, 1),
 (1, 275, 1),
 (1, 279, 2),
 (1, 281, 1),
 (1, 286

## Part 2: Preparing Sparse Matrix data for NMF 


We will use the [coo matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01) function to turn the sparse matrix into an array. 


In [26]:
import numpy as np

from scipy.sparse import coo_matrix

rows = [x[0] for x in sparsemat]
cols = [x[1] for x in sparsemat]
values = [x[2] for x in sparsemat]

coo = coo_matrix((values, (rows, cols)))

In [27]:
# print(coo)

In [33]:
import numpy as np

from scipy.sparse import coo_matrix

rows1 = [x[1]-1 for x in sparsemat]
cols1 = [x[0]-1 for x in sparsemat]
values1 = [x[2] for x in sparsemat]

coo1 = coo_matrix((values1, (rows1, cols1)))

In [34]:
with urllib.request.urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/bbc.terms') as r:
    content = r.readlines()

words = [c.split()[0] for c in content]


In [35]:
with urllib.request.urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/bbc.docs') as r:
    content = r.readlines()

docs = [c.split()[0] for c in content]

In [36]:
print(len(coo1.toarray()), len(words), len(docs))

2225 9635 2225


In [37]:
import pandas as pd

pd.DataFrame(coo1.toarray(), columns= words, index= docs)

,b'ad',b'sale',b'boost',b'time',b'warner',b'profit',b'quarterli',b'media',b'giant',b'jump',...,b'\xc2\xa3339',b'denialofservic',b'ddo',b'seagrav',b'bot',b'wirelessli',b'streamcast',b'peripher',b'headphon',b'flavour'
b'business.001',1,5,2,3,4,10,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
b'business.002',0,0,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b'business.003',0,4,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
b'business.004',0,1,0,0,0,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b'business.005',0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b'tech.397',0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b'tech.398',0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
b'tech.399',0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
b'tech.400',0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## NMF


NMF is a way of decomposing a matrix of documents and words so that one of the matrices can be interpreted as the "loadings" or "weights" of each word on a topic. 


Check out [the NMF documentation](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01) and the [examples of topic extraction using NMF and LDA](http://scikit-learn.org/0.18/auto_examples/applications/topics_extraction_with_nmf_lda.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01).


## Part 3

Here, we will import `NMF`, define a model object with 5 components, and `fit_transform` the data created above.


In [61]:
# Surpress warnings from using older version of sklearn:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from sklearn.decomposition import NMF

model = NMF(n_components=5, init='random', random_state=818)
doc_topic = model.fit_transform(coo)

doc_topic.shape
# we should have 9636 observations (articles) and five latent features

(9636, 5)

In [62]:
doc_topic

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [2.43423348e+00, 7.01529551e-01, 1.03935248e+00, 5.39638492e-01,
        4.99462408e-01],
       [0.00000000e+00, 5.49866061e-01, 2.44845915e+00, 3.27169531e-01,
        2.97735273e-01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 2.48510074e-03, 1.60218523e-03,
        2.10108039e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.72993384e-03,
        2.00619076e-02],
       [0.00000000e+00, 0.00000000e+00, 1.58662286e-03, 0.00000000e+00,
        1.46731272e-02]])

In [63]:
# find feature with highest value per doc
np.argmax(doc_topic, axis=1)

array([0, 0, 2, ..., 4, 4, 4])

## Part 4: 

Check out the `components` of this model:


In [64]:
model.components_.shape

(5, 2226)

In [70]:
model

NMF(alpha=0.0, beta_loss='frobenius', init='random', l1_ratio=0.0,
  max_iter=200, n_components=5, random_state=818, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

This is five rows, each of which is a "topic" containing the weights of each word on that topic. The exercise is to _get a list of the top 10 words for each topic_. We can just store this in a list of lists.


**Note:** Just like we read in the data above, we'll have to read in the words from the `bbc.terms` file.


In [71]:
with urllib.request.urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/bbc.terms') as r:
    content = r.readlines()
words = [c.split()[0] for c in content]

In [72]:
len(words)

9635

In [67]:
words[0:10]

[b'ad',
 b'sale',
 b'boost',
 b'time',
 b'warner',
 b'profit',
 b'quarterli',
 b'media',
 b'giant',
 b'jump']

In [68]:
topic_words = []
for r in model.components_:
    a = sorted([(v,i) for i,v in enumerate(r)],reverse=True)[0:12]
    topic_words.append([words[e[1]] for e in a])

In [69]:
# Here, each set of words relates to the corresponding topic (ie the first set of words relates to topic 'Business', etc.)
topic_words[:5]

[[b'bondi',
  b'stanlei',
  b'continent',
  b'mortgag',
  b'bare',
  b'least',
  b'extent',
  b'200',
  b'leav',
  b'frustrat',
  b'yuan',
  b'industri'],
 [b'manipul',
  b'teenag',
  b'drawn',
  b'go',
  b'prosecutor',
  b'herbert',
  b'host',
  b'protest',
  b'hike',
  b'nation',
  b'calcul',
  b'power'],
 [b'dimens',
  b'hous',
  b'march',
  b'wider',
  b'owner',
  b'intend',
  b'declin',
  b'forc',
  b'posit',
  b'founder',
  b'york',
  b'unavail'],
 [b'rome',
  b'ft',
  b'regain',
  b'lawmak',
  b'outright',
  b'resum',
  b'childhood',
  b'greatest',
  b'citi',
  b'stagnat',
  b'crown',
  b'bodi'],
 [b'build',
  b'empir',
  b'isol',
  b'\xc2\xa312',
  b'restructur',
  b'closer',
  b'plung',
  b'depreci',
  b'durham',
  b'race',
  b'juli',
  b'segreg']]

The original data had 5 topics, as listed in `bbc.docs` (which these topic words relate to). 

```
Business
Entertainment
Politics
Sport
Tech
```

In "real life", we would have found a way to use these to inform the model. But for this little demo, we can just compare the recovered topics to the original ones. And they seem to match reasonably well. The order is different, which is to be expected in this kind of model.


In [25]:
with urllib.request.urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/bbc.docs') as r:
    doc_content = r.readlines()
    
doc_content[:8]

[b'business.001\n',
 b'business.002\n',
 b'business.003\n',
 b'business.004\n',
 b'business.005\n',
 b'business.006\n',
 b'business.007\n',
 b'business.008\n']

In [58]:
# doc_content

---
### Machine Learning Foundation (C) 2020 IBM Corporation
